In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import os
import zipfile
import shutil
from PIL import Image
import numpy as np
import pandas as pd
import time
import dask.bag as db
import dask.diagnostics as dd
import random
from termcolor import colored
import tensorflow as tf
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2M, EfficientNetV2B0, EfficientNetV2B3, Xception, ResNet50
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import Precision, Recall, F1Score

2024-11-19 15:41:49.008253: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
!nvidia-smi

Tue Nov 19 15:41:51 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  |   00000000:4E:00.0 Off |                   On |
| N/A   28C    P0             49W /  400W |                  N/A   |     N/A      Default |
|                                         |                        |              Enabled |
+-----------------------------------------+-----

In [3]:
CWD = os.getcwd()
CWD

'/workspace'

In [4]:
EPOCHS = 30
SAVE_DIRECTORY = CWD + '/model'

# Load Data

In [ ]:
BASE_DIR = os.path.join(CWD, 'dataset', 'processed', 'Training')
TEST_DIR = os.path.join(CWD, 'dataset', 'processed', 'Testing')
CLASS_DIR = os.path.join(CWD, 'train_300', 'Training')

In [6]:
# Loop over folders to extract class_names
classes = [class_name for class_name in os.listdir(CLASS_DIR) if os.path.isdir(os.path.join(CLASS_DIR, class_name)) and not class_name.startswith('.')]
classes

['glioma', 'meningioma', 'pituitary', 'notumor']

## Prepare Dataset

In [7]:
def prepare_dataset(BASE_DIR, IMG_SIZE, batch_size):
    train_full = tf.keras.utils.image_dataset_from_directory(
        directory=BASE_DIR,        
        labels='inferred',         
        label_mode='categorical',  
        class_names=classes,       
        seed=42,                   
        batch_size=batch_size,             
        image_size=(IMG_SIZE, IMG_SIZE)      
    )

    train_full = train_full.prefetch(tf.data.AUTOTUNE)
    
    # Counting number of all batches in dataset
    num_of_full_train_batches = len(list(train_full))
    print(colored(f'Number of batches in train_full : {num_of_full_train_batches}', 'red', attrs=['bold']))
    
    # Define variable to store number of batches for train dataset
    num_train_batches = int(num_of_full_train_batches * 0.70)
    
    # Define variable to store number of batches for validation and test dataset
    num_valid_test_batches = num_of_full_train_batches - num_train_batches
    
    # Print the TARGET : number of batches for train, validation and test dataset to each
    print(colored(' Target : ', 'green', attrs=['bold']))
    print('-'*35)
    print(colored(f'Number of  Train  batches : {num_train_batches}', 'blue', attrs=['bold']))
    print(colored(f'Number of Validation batches : {num_valid_test_batches//2}', 'blue', attrs=['bold']))
    print(colored(f'Number of Test batches : {num_valid_test_batches//2}', 'blue', attrs=['bold']))

    # Apply above settings to main dataset to split to train, validation and test dataset
    train_ds = train_full.take(num_train_batches)
    remain_ds = train_full.skip(num_train_batches)
    valid_ds = remain_ds.take(num_valid_test_batches)

    train_ds = train_ds.shuffle(buffer_size=3)
    

    test_ds = tf.keras.utils.image_dataset_from_directory(
        directory=TEST_DIR,
        labels='inferred',
        label_mode = 'categorical',
        class_names=classes,
        seed=42,
        batch_size=batch_size,
        image_size=(IMG_SIZE, IMG_SIZE)
    )

    return train_ds, valid_ds, test_ds

In [8]:
def prepare_dataset(BASE_DIR, TEST_DIR, IMG_SIZE, batch_size):
    train_full = tf.keras.utils.image_dataset_from_directory(
        directory=BASE_DIR,        
        labels='inferred',         
        label_mode='categorical',  
        class_names=classes,       
        seed=42,                   
        batch_size=batch_size,             
        image_size=(IMG_SIZE, IMG_SIZE)      
    )

    train_full = train_full.prefetch(tf.data.AUTOTUNE)
    
    # Counting number of all batches in dataset
    num_of_full_train_batches = len(list(train_full))
    print(colored(f'Number of batches in train_full : {num_of_full_train_batches}', 'red', attrs=['bold']))
    
    # Define variable to store number of batches for train dataset
    num_train_batches = int(num_of_full_train_batches * 0.70)
    
    # Define variable to store number of batches for validation and test dataset
    num_valid_test_batches = num_of_full_train_batches - num_train_batches
    
    # Print the TARGET : number of batches for train, validation and test dataset to each
    print(colored(' Target : ', 'green', attrs=['bold']))
    print('-'*35)
    print(colored(f'Number of  Train  batches : {num_train_batches}', 'blue', attrs=['bold']))
    print(colored(f'Number of Validation batches : {num_valid_test_batches//2}', 'blue', attrs=['bold']))
    print(colored(f'Number of Test batches : {num_valid_test_batches//2}', 'blue', attrs=['bold']))

    # Apply above settings to main dataset to split to train, validation and test dataset
    train_ds = train_full.take(num_train_batches)
    valid_ds = train_full.take(num_valid_test_batches)

    train_ds = train_ds.shuffle(buffer_size=3)
    

    test_ds = tf.keras.utils.image_dataset_from_directory(
        directory=TEST_DIR,
        labels='inferred',
        label_mode = 'categorical',
        class_names=classes,
        seed=42,
        batch_size=batch_size,
        image_size=(IMG_SIZE, IMG_SIZE)
    )

    return train_ds, valid_ds, test_ds

## Model Creation

### Not Trainable

In [ ]:
def build_model_not_trainable(base_model, input_shape, num_classes):
    input = tf.keras.layers.Input(shape=input_shape)
    
    base_model = base_model(input_shape=input_shape, include_top=False, classes=num_classes, weights='imagenet')

    # Freeze all layers
    base_model.trainable = False

    x = base_model(input, training=False)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)               
    x = tf.keras.layers.Dense(128, activation='relu')(x)

    output = tf.keras.layers.Dense(4, activation='softmax')(x)

    model = tf.keras.Model(inputs=input, outputs=output)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy', F1Score()]
    )

    return model

### First Part

In [ ]:
def build_model_trainable_first25(base_model, input_shape, num_classes):
    input = tf.keras.layers.Input(shape=input_shape)
    
    base_model = base_model(input_shape=input_shape, include_top=False, classes=num_classes, weights='imagenet')
    base_model.trainable = True

    # Freeze last 75% layers
    total_layers = len(base_model.layers)
    for layer in base_model.layers[(int(0.75 * total_layers)):]:
        layer.trainable = False

    x = base_model(input, training=False)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)               
    x = tf.keras.layers.Dense(128, activation='relu')(x)

    output = tf.keras.layers.Dense(4, activation='softmax')(x)

    model = tf.keras.Model(inputs=input, outputs=output)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy', F1Score()]
    )

    return model  

In [ ]:
def build_model_trainable_first50(base_model, input_shape, num_classes):
    input = tf.keras.layers.Input(shape=input_shape)
    
    base_model = base_model(input_shape=input_shape, include_top=False, classes=num_classes, weights='imagenet')
    base_model.trainable = True

    # Freeze last 50% layers
    total_layers = len(base_model.layers)
    for layer in base_model.layers[(int(0.50 * total_layers)):]:
        layer.trainable = False

    x = base_model(input, training=False)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)               
    x = tf.keras.layers.Dense(128, activation='relu')(x)

    output = tf.keras.layers.Dense(4, activation='softmax')(x)

    model = tf.keras.Model(inputs=input, outputs=output)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy', F1Score()]
    )

    return model  

In [ ]:
def build_model_trainable_first75(base_model, input_shape, num_classes):
    input = tf.keras.layers.Input(shape=input_shape)
    
    base_model = base_model(input_shape=input_shape, include_top=False, classes=num_classes, weights='imagenet')
    base_model.trainable = True

    # Freeze last 25% layers
    total_layers = len(base_model.layers)
    for layer in base_model.layers[(int(0.25 * total_layers)):]:
        layer.trainable = False

    x = base_model(input, training=False)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)               
    x = tf.keras.layers.Dense(128, activation='relu')(x)

    output = tf.keras.layers.Dense(4, activation='softmax')(x)

    model = tf.keras.Model(inputs=input, outputs=output)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy', F1Score()]
    )

    return model  

### Last Part

In [9]:
def build_model_trainable_last25(base_model, input_shape, num_classes):
    input = tf.keras.layers.Input(shape=input_shape)
    
    base_model = base_model(input_shape=input_shape, include_top=False, classes=num_classes, weights='imagenet')
    base_model.trainable = True

    # Freeze first 75% layers
    total_layers = len(base_model.layers)
    for layer in base_model.layers[:(int(0.75 * total_layers))]:
        layer.trainable = False

    x = base_model(input, training=False)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)               
    x = tf.keras.layers.Dense(128, activation='relu')(x)

    output = tf.keras.layers.Dense(4, activation='softmax')(x)

    model = tf.keras.Model(inputs=input, outputs=output)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy', F1Score()]
    )

    return model    

In [ ]:
def build_model_trainable_last50(base_model, input_shape, num_classes):
    input = tf.keras.layers.Input(shape=input_shape)
    
    base_model = base_model(input_shape=input_shape, include_top=False, classes=num_classes, weights='imagenet')
    base_model.trainable = True

    # Freeze first 50% layers
    total_layers = len(base_model.layers)
    for layer in base_model.layers[:(int(0.50 * total_layers))]:
        layer.trainable = False

    x = base_model(input, training=False)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)               
    x = tf.keras.layers.Dense(128, activation='relu')(x)

    output = tf.keras.layers.Dense(4, activation='softmax')(x)

    model = tf.keras.Model(inputs=input, outputs=output)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy', F1Score()]
    )

    return model    

In [ ]:
def build_model_trainable_last75(base_model, input_shape, num_classes):
    input = tf.keras.layers.Input(shape=input_shape)
    
    base_model = base_model(input_shape=input_shape, include_top=False, classes=num_classes, weights='imagenet')
    base_model.trainable = True

    # Freeze first 25% layers
    total_layers = len(base_model.layers)
    for layer in base_model.layers[:(int(0.25 * total_layers))]:
        layer.trainable = False

    x = base_model(input, training=False)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)               
    x = tf.keras.layers.Dense(128, activation='relu')(x)

    output = tf.keras.layers.Dense(4, activation='softmax')(x)

    model = tf.keras.Model(inputs=input, outputs=output)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy', F1Score()]
    )

    return model    

### Full Trainable

In [ ]:
def build_model_full_trainable(base_model, input_shape, num_classes):
    input = tf.keras.layers.Input(shape=input_shape)
    
    base_model = base_model(input_shape=input_shape, include_top=False, classes=num_classes, weights='imagenet')

    # Full Trainable
    base_model.trainable = True

    x = base_model(input, training=False)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)               
    x = tf.keras.layers.Dense(128, activation='relu')(x)

    output = tf.keras.layers.Dense(4, activation='softmax')(x)

    model = tf.keras.Model(inputs=input, outputs=output)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy', F1Score()]
    )

    return model    

## Train Function

In [12]:
def train(model, model_name, train_ds, valid_ds, epochs, save_directory):
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)

    saving_path = os.path.join(save_directory, f'model_{model_name}.h5')

    # Model CheckPoint Call-Back, to save best model parameters as a .keras file
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(saving_path, monitor='val_accuracy', save_best_only=True)

    # Early Stoping Call-Backc to stop trainig process after 'patience' epochs if the metric doesn't grow
    #earlystop_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

    # ReduceLROnPlateau Call-Back to decrease learning-rate base on 'monitor' parameter after 'patience' epochs with a 'factor' is doesn't improve
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

    start = time.time()

    # Model training
    history = model.fit(
        train_ds,
        epochs=epochs,
        validation_data = valid_ds,
        callbacks=[checkpoint_cb, reduce_lr]
    )

    end = time.time()

    # Counting trianing time
    training_time = end - start
    print(f"Training completed in {training_time:.2f} seconds.")

    return model, history, training_time

## Best Model Selection Function

In [ ]:
def best_variant(models, test_ds, output_csv, output_folder='results'):
    """
    Compare multiple models, select the best based on F1 Score, and save results to a CSV.

    Args:
        models (list of tuples): List of (model_name, model, history, time) tuples.
        test_ds: The test dataset for evaluation.
        output_csv (str): The name of the output CSV file.
        output_folder (str): The directory to save the results.
        
    Returns:
        Tuple containing the best model's name, test loss, test accuracy, test F1 score, and training time.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Evaluate each model and collect results
    models_info = []

    for model_name, model, history, time in models:
        # Assume val_accuracy as a proxy for F1 score
        val_f1score = max(history.history['val_accuracy'])
        val_loss = min(history.history['val_loss'])
        val_acc = max(history.history['val_accuracy'])

        # Store each model's information in models_info
        models_info.append({
            'Model Name': model_name,
            'Model': model,
            'Loss': val_loss,
            'Accuracy': val_acc,
            'F1 Score': val_f1score,
            'Training Time': time,
            'History': history
        })

    # Sort the models by F1 Score in descending order
    models_info = sorted(models_info, key=lambda x: x['F1 Score'], reverse=True)

    # Convert to DataFrame for a neat tabular display
    df = pd.DataFrame([{k: v for k, v in m.items() if k != 'Model' and k != 'History'} for m in models_info])

    # Save to CSV
    output_path = os.path.join(output_folder, output_csv)
    df.to_csv(output_path, index=False)
    print(f"Model comparison table saved to {output_path}")

    # Print DataFrame
    print("Model Comparison Table:")
    print(df)

    # Select the best model based on F1 Score
    best_model_info = models_info[0]
    best_model = best_model_info['Model']
    best_model_name = best_model_info['Model Name']
    best_history = best_model_info['History']
    best_time = best_model_info['Training Time']

    # Evaluate on the test dataset using plot function
    best_history, test_loss, test_acc, test_f1score = plot(best_model, best_history, test_ds, f"results/{best_model_name}",  best_model_name)

    # Return the best model's evaluation results for updating result_dict later
    return best_model_name, test_loss, test_acc, test_f1score, best_time

## Plot Function

In [ ]:
def plot(model, history, test_ds, output_folder='results', prefix='model'):
    """
    Plot training results, confusion matrix, and save plots to files.

    Args:
        model: The trained model.
        history: Training history object from model.fit().
        test_ds: Test dataset for evaluation.
        output_folder (str): Directory to save plots.
        prefix (str): Prefix for output file names.

    Returns:
        Tuple containing history, test loss, test accuracy, and test F1 score.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Convert result of training to DataFrame
    result = pd.DataFrame(history.history)

    # Define a variable to store range of epochs
    x = np.arange(len(result))

    # Create a plot with 2 rows and 1 column with size of (15, 12)
    fig, ax = plt.subplots(2, 1, figsize=(15, 12))

    # AX0 : Loss
    ax[0].plot(x, result.loss, label='Loss', linewidth=3)
    ax[0].plot(x, result.val_loss, label='Validation Loss', linewidth=2)
    ax[0].set_title('Loss', fontsize=20)
    ax[0].set_xticks(np.arange(0, len(x), 2))
    ax[0].legend()

    # AX1 : Accuracy
    ax[1].plot(x, result.accuracy, label='Accuracy', linewidth=2)
    ax[1].plot(x, result.val_accuracy, label='Validation Accuracy', linewidth=2)
    ax[1].set_title('Accuracy', fontsize=20)
    ax[1].set_xticks(np.arange(0, len(x), 2))
    ax[1].legend()

    # Save accuracy and loss plot
    acc_loss_plot_path = os.path.join(output_folder, f"{prefix}_acc_loss_plot.png")
    plt.savefig(acc_loss_plot_path)
    print(f"Accuracy and Loss plot saved to {acc_loss_plot_path}")

    # Evaluate model
    test_loss, test_acc, test_f1score = model.evaluate(test_ds, verbose=1)

    # Confusion Matrix
    y_pred = []
    y_true = []

    for images, labels in test_ds:
        predictions = model.predict(images)
        y_pred.extend(np.argmax(predictions, axis=1))

        # If labels are one-hot encoded, convert to class indices
        if len(labels.shape) > 1:
            labels = np.argmax(labels, axis=1)
        y_true.extend(labels)

    # Create confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    # Get the class names from the dataset
    class_names = sorted(set(y_true))  # Infer class names from true labels

    # Display the confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap=plt.cm.Blues)

    # Save confusion matrix plot
    cm_plot_path = os.path.join(output_folder, f"{prefix}_confusion_matrix.png")
    plt.savefig(cm_plot_path)
    print(f"Confusion Matrix plot saved to {cm_plot_path}")

    # Classification Report
    print('Classification Report')
    cr = classification_report(y_true, y_pred, digits=5, target_names=[str(c) for c in class_names])
    print(cr)

    # Save classification report to a text file
    cr_path = os.path.join(output_folder, f"{prefix}_classification_report.txt")
    with open(cr_path, 'w') as f:
        f.write('Classification Report\n')
        f.write(cr)
    print(f"Classification Report saved to {cr_path}")

    # Show all plots
    plt.show()

    return history, test_loss, test_acc, test_f1score

In [ ]:
def save_results_to_csv(result_dict, file_path='results/results.csv'):
    # Convert result_dict to DataFrame
    df = pd.DataFrame(result_dict)
    
    # Save the combined DataFrame to CSV
    df.to_csv(file_path, index=False)
    print(f"Results saved to {file_path}")

# Function to load results from a CSV file
def load_results_from_csv(file_path='results/result.csv'):
    # If the file exists, load it; otherwise, return an empty dictionary
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        result_dict =  df.to_dict(orient='list')
    else:
        result_dict =  {
            'Model Name': [],
            'Test Loss': [],
            'Test Accuracy': [],
            'Test F1Score': [],
            'Training Time': []
        }
    return result_dict

In [16]:
def result(model_name, test_loss, test_acc, test_f1score, result_dict, training_time):
    
    test_f1score = test_f1score[1] if isinstance(test_f1score, (list, np.ndarray)) else test_f1score
    
    result_dict['Model Name'].append(model_name)
    result_dict['Test Loss'].append(test_loss)
    result_dict['Test Accuracy'].append(test_acc)
    result_dict['Test F1Score'].append(test_f1score)
    result_dict['Training Time'].append(training_time)
    
    save_results_to_csv(result_dict, 'results/result.csv')

result_dict = {
    'Model Name': [],
    'Test Loss': [],
    'Test Accuracy': [],
    'Test F1Score': [],
    'Training Time': []
}

# Prepare training dataset

In [ ]:
train_dir_300 = "train_300/Training"
test_dir_300= "train_300/Testing"

In [ ]:
train_ds_300, valid_ds_300, test_ds_300 = prepare_dataset(train_dir_300, test_dir_300, 300, 16)

Found 5712 files belonging to 4 classes.


2024-11-19 15:42:16.402001: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1636] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 17947 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB MIG 3g.20gb, pci bus id: 0000:4e:00.0, compute capability: 8.0


Number of batches in train_full : 179
 Target : 
-----------------------------------
Number of  Train  batches : 125
Number of Validation batches : 27
Number of Test batches : 27
Found 1311 files belonging to 4 classes.
Found 5712 files belonging to 4 classes.
Number of batches in train_full : 357
 Target : 
-----------------------------------
Number of  Train  batches : 249
Number of Validation batches : 54
Number of Test batches : 54
Found 1311 files belonging to 4 classes.
Found 5712 files belonging to 4 classes.
Number of batches in train_full : 2856
 Target : 
-----------------------------------
Number of  Train  batches : 1999
Number of Validation batches : 428
Number of Test batches : 428
Found 1311 files belonging to 4 classes.


# Model Training

## Standard

### EfficientNetV2B0

#### not trainable

In [ ]:
model_name_1 = 'EfficientnetV2B0 Without Augmentation Not Trainable'
input_shape = (300,300,3)

std_b0_nt = build_model_not_trainable(EfficientNetV2B0, input_shape=input_shape, num_classes=4)
std_b0_nt, history_std_b0_nt, training_time_1 = train(std_b0_nt, model_name_1, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

#### trainable first 25% layers

In [ ]:
model_name_2 = "EfficientnetV2B0 Without Augmentation Trainable First 25% Layer"
input_shape = (300,300,3)

std_b0_tf25 = build_model_trainable_first25(EfficientNetV2B0, input_shape=input_shape, num_classes=4)
std_b0_tf25, history_std_b0_tf25, training_time_2 = train(std_b0_tf25, model_name_2, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

#### trainable first 50% layers

In [ ]:
model_name_3 = 'EfficientnetV2B0 Without Augmentation Trainable First 50% Layer'
input_shape = (300,300,3)

std_b0_tf50 = build_model_trainable_last50(EfficientNetV2B0, input_shape=input_shape, num_classes=4)
std_b0_tf50, history_std_b0_tf50, training_time_3 = train(std_b0_tf50, model_name_3, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

#### trainable first 75% layers

In [ ]:
model_name_4 = 'EfficientnetV2B0 Without Augmentation Trainable First 75% Layer'
input_shape = (300,300,3)

std_b0_tf75 = build_model_trainable_last75(EfficientNetV2B0, input_shape=input_shape, num_classes=4)
std_b0_tf75, history_std_b0_tf75, training_time_4 = train(std_b0_tf75, model_name_4, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

#### trainable last 25% layers

In [ ]:
model_name_5 = 'EfficientnetV2B0 Without Augmentation Trainable Last 25% Layer'
input_shape = (300,300,3)

std_b0_tl25 = build_model_trainable_last25(EfficientNetV2B0, input_shape=input_shape, num_classes=4)
std_b0_tl25, history_std_b0_tl25, training_time_5 = train(std_b0_tl25, model_name_5, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

#### trainable last 50% layers

In [ ]:
model_name_6 = 'EfficientnetV2B0 Without Augmentation Trainable Last 50% Layer'
input_shape = (300,300,3)

std_b0_tl50 = build_model_trainable_last50(EfficientNetV2B0, input_shape=input_shape, num_classes=4)
std_b0_tl50, history_std_b0_tl50, training_time_6 = train(std_b0_tl50, model_name_6, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

#### trainable last 75% layers

In [ ]:
model_name_7 = 'EfficientnetV2B0 Without Augmentation Trainable Last 75% Layer'
input_shape = (300,300,3)

std_b0_tl75 = build_model_trainable_last75(EfficientNetV2B0, input_shape=input_shape, num_classes=4)
std_b0_tl75, history_std_b0_tl75, training_time_7 = train(std_b0_tl75, model_name_7, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

#### full trainable

In [ ]:
model_name_8 = 'EfficientnetV2B0 Without Augmentation Full Trainable'
input_shape = (300,300,3)

std_b0_ft = build_model_full_trainable(EfficientNetV2B0, input_shape=input_shape, num_classes=4)
std_b0_ft, history_std_b0_ft, training_time_8 = train(std_b0_ft, model_name_8, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

In [ ]:
models = [
    (model_name_1, std_b0_nt, history_std_b0_nt, training_time_1),
    (model_name_2, std_b0_tf25, history_std_b0_tf25, training_time_2),
    (model_name_3, std_b0_tf50, history_std_b0_tf50, training_time_3),
    (model_name_4, std_b0_tf75, history_std_b0_tf75, training_time_4),
    (model_name_5, std_b0_tl25, history_std_b0_tl25, training_time_5),
    (model_name_6, std_b0_tl50, history_std_b0_tl50, training_time_6),
    (model_name_7, std_b0_tl75, history_std_b0_tl75, training_time_7),
    (model_name_8, std_b0_ft, history_std_b0_ft, training_time_8)
]

best_model_name, test_loss, test_acc, test_f1score, best_time = best_variant(
    models=models,
    test_ds=test_ds_300,
    output_csv='EfficientNetV2B0 Without Agumentation.csv',
    output_folder='results'
)

config_1 = best_model_name

In [24]:
result(best_model_name, test_loss, test_acc, test_f1score, result_dict, best_time)

# Convert result_dict to a DataFrame
df = pd.DataFrame(result_dict)

# Convert any TensorFlow tensors in the F1Score column to numpy for better readability
df['Test F1Score'] = df['Test F1Score'].apply(lambda x: x.numpy() if hasattr(x, 'numpy') else x)

# Print the DataFrame as a table
print("Result Dictionary Table:")
print(df)

Results saved to results/result.csv
Result Dictionary Table:
                                          Model Name  Test Loss  \
0  EfficientnetV2B0 Without Augmentation Trainabl...   0.092373   

   Test Accuracy  Test F1Score  Training Time  
0       0.985507      0.972536     284.886228  


### EfficientNetV2B3

#### not trainable

In [ ]:
model_name_1 = 'EfficientnetV2B3 Without Augmentation Not Trainable'
input_shape = (300,300,3)

std_b3_nt = build_model_not_trainable(EfficientNetV2B3, input_shape=input_shape, num_classes=4)
std_b3_nt, history_std_b3_nt, training_time_1 = train(std_b3_nt, model_name_1, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

#### trainable first 25% layers

In [ ]:
model_name_2 = 'EfficientnetV2B3 Without Augmentation Trainable First 25% Layer'
input_shape = (300,300,3)

std_b3_tf25 = build_model_trainable_first25(EfficientNetV2B3, input_shape=input_shape, num_classes=4)
std_b3_tf25, history_std_b3_tf25, training_time_2 = train(std_b3_tf25, model_name_2, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

#### trainable first 50% layers

In [ ]:
model_name_3 = 'EfficientnetV2B3 Without Augmentation Trainable First 50% Layer'
input_shape = (300,300,3)

std_b3_tf50 = build_model_trainable_first50(EfficientNetV2B3, input_shape=input_shape, num_classes=4)
std_b3_tf50, history_std_b3_tf50, training_time_3 = train(std_b3_tf50, model_name_3, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

#### trainable first 75% layers

In [ ]:
model_name_4 = 'EfficientnetV2B3 Without Augmentation Trainable First 75% Layer'
input_shape = (300,300,3)

std_b3_tf75 = build_model_trainable_first75(EfficientNetV2B3, input_shape=input_shape, num_classes=4)
std_b3_tf75, history_std_b3_tf75, training_time_4 = train(std_b3_tf75, model_name_4, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

#### trainable last 25% layers

In [ ]:
model_name_5 = 'EfficientnetV2B3 Without Augmentation Trainable Last 25% Layer'
input_shape = (300,300,3)

std_b3_tl25 = build_model_trainable_last25(EfficientNetV2B3, input_shape=input_shape, num_classes=4)
std_b3_tl25, history_std_b3_tl25, training_time_5 = train(std_b3_tl25, model_name_5, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

#### trainable last 50% layers

In [ ]:
model_name_6 = 'EfficientnetV2B3 Without Augmentation Trainable Last 50% Layer'
input_shape = (300,300,3)

std_b3_tl50 = build_model_trainable_last50(EfficientNetV2B3, input_shape=input_shape, num_classes=4)
std_b3_tl50, history_std_b3_tl50, training_time_6 = train(std_b3_tl50, model_name_6, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

#### trainable last 75% layers

In [ ]:
model_name_7 = 'EfficientnetV2B3 Without Augmentation Trainable Last 75% Layer'
input_shape = (300,300,3)

std_b3_tl75 = build_model_trainable_last75(EfficientNetV2B3, input_shape=input_shape, num_classes=4)
std_b3_tl75, history_std_b3_tl75, training_time_7 = train(std_b3_tl75, model_name_7, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

#### full trainable

In [ ]:
model_name_8 = 'EfficientnetV2B3 Without Augmentation Trainable Full Trainable'
input_shape = (300,300,3)

std_b3_ft = build_model_full_trainable(EfficientNetV2B3, input_shape=input_shape, num_classes=4)
std_b3_ft, history_std_b3_ft, training_time_8 = train(std_b3_ft, model_name_8, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

In [ ]:
models = [
    (model_name_1, std_b3_nt, history_std_b3_nt, training_time_1),
    (model_name_2, std_b3_tf25, history_std_b3_tf25, training_time_2),
    (model_name_3, std_b3_tf50, history_std_b3_tf50, training_time_3),
    (model_name_4, std_b3_tf75, history_std_b3_tf75, training_time_4),
    (model_name_5, std_b3_tl25, history_std_b3_tl25, training_time_5),
    (model_name_6, std_b3_tl50, history_std_b3_tl50, training_time_6),
    (model_name_7, std_b3_tl75, history_std_b3_tl75, training_time_7),
    (model_name_8, std_b3_ft, history_std_b3_ft, training_time_8)
]

best_model_name, test_loss, test_acc, test_f1score, best_time = best_variant(
    models=models,
    test_ds=test_ds_300,
    output_csv='EfficientNetV2B3 Without Agumentation.csv',
    output_folder='results'
)

config_2 = best_model_name

In [29]:
result(best_model_name, test_loss, test_acc, test_f1score, result_dict, best_time)

# Convert result_dict to a DataFrame
df = pd.DataFrame(result_dict)

# Convert any TensorFlow tensors in the F1Score column to numpy for better readability
df['Test F1Score'] = df['Test F1Score'].apply(lambda x: x.numpy() if hasattr(x, 'numpy') else x)

# Print the DataFrame as a table
print("Result Dictionary Table:")
print(df)

Results saved to results/result.csv
Result Dictionary Table:
                                          Model Name  Test Loss  \
0  EfficientnetV2B0 Without Augmentation Trainabl...   0.092373   
1  EfficientnetV2B3 Without Augmentation Trainabl...   0.053577   

   Test Accuracy  Test F1Score  Training Time  
0       0.985507      0.972536     284.886228  
1       0.992372      0.987055     818.727502  


### EfficientNetV2M

#### not trainable

In [ ]:
model_name_1 = 'EfficientnetV2M Withouth Augmentation Not Trainable'
input_shape = (300,300,3)

std_m_nt = build_model_not_trainable(EfficientNetV2M, input_shape=input_shape, num_classes=4)
std_m_nt, history_std_m_nt, training_time_1 = train(std_m_nt, model_name_1, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

#### trainable first 25% layers

In [ ]:
model_name_2 = 'EfficientnetV2M Without Augmentation Trainable First 25% Layer'
input_shape = (300,300,3)

std_m_tf25 = build_model_trainable_first25(EfficientNetV2M, input_shape=input_shape, num_classes=4)
std_m_tf25, history_std_m_tf25, training_time_2 = train(std_m_tf25, model_name_2, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

#### trainable first 50% layers

In [ ]:
model_name_3 = 'EfficientnetV2M Without Augmentation Trainable First 50% Layer'
input_shape = (300,300,3)

std_m_tf50 = build_model_trainable_first50(EfficientNetV2M, input_shape=input_shape, num_classes=4)
std_m_tf50, history_std_m_tf50, training_time_3 = train(std_m_tf50, model_name_3, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

#### trainable first 75% layers

In [ ]:
model_name_4 = 'EfficientnetV2M Without Augmentation Trainable First 75% Layer'
input_shape = (300,300,3)

std_m_tf75 = build_model_trainable_first75(EfficientNetV2M, input_shape=input_shape, num_classes=4)
std_m_tf75, history_std_m_tf75, training_time_4 = train(std_m_tf75, model_name_4, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

#### trainable last 25% layers

In [ ]:
model_name_5 = 'EfficientnetV2M Without Augmentation Trainable Last 25% Layer'
input_shape = (300,300,3)

std_m_tl25 = build_model_trainable_last25(EfficientNetV2M, input_shape=input_shape, num_classes=4)
std_m_tl25, history_std_m_tl25, training_time_5 = train(std_m_tl25, model_name_5, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

#### trainable last 50% layers

In [ ]:
model_name_6 = 'EfficientnetV2M Without Augmentation Trainable Last 50% Layer'
input_shape = (300,300,3)

std_m_tl50 = build_model_trainable_last50(EfficientNetV2M, input_shape=input_shape, num_classes=4)
std_m_tl50, history_std_m_tl50, training_time_6 = train(std_m_tl50, model_name_6, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

#### trainable last 75% layers

In [ ]:
model_name_7 = 'EfficientnetV2M Without Augmentation Trainable Last 75% Layer'
input_shape = (300,300,3)

std_m_tl75 = build_model_trainable_last75(EfficientNetV2M, input_shape=input_shape, num_classes=4)
std_m_tl75, history_std_m_tl75, training_time_7 = train(std_m_tl75, model_name_7, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

#### full trainable

In [ ]:
model_name_8 = 'EfficientnetV2M Without Augmentation Full Trainable'
input_shape = (300,300,3)

std_m_ft = build_model_full_trainable(EfficientNetV2M, input_shape=input_shape, num_classes=4)
std_m_ft, history_std_m_ft, training_time_8 = train(std_m_ft, model_name_8, train_ds_300, valid_ds_300, EPOCHS, SAVE_DIRECTORY)

In [ ]:
models = [
    (model_name_1, std_m_nt, history_std_m_nt, training_time_1),
    (model_name_2, std_m_tf25, history_std_m_tf25, training_time_2),
    (model_name_3, std_m_tf50, history_std_m_tf50, training_time_3),
    (model_name_4, std_m_tf75, history_std_m_tf75, training_time_4),
    (model_name_5, std_m_tl25, history_std_m_tl25, training_time_5),
    (model_name_6, std_m_tl50, history_std_m_tl50, training_time_6),
    (model_name_7, std_m_tl75, history_std_m_tl75, training_time_7),
    (model_name_8, std_m_ft, history_std_m_ft, training_time_8)
]

best_model_name, test_loss, test_acc, test_f1score, best_time = best_variant(
    models=models,
    test_ds=test_ds_300,
    output_csv='EfficientNetV2M Without Agumentation.csv',
    output_folder='results'
)

config_3 = best_model_name

In [35]:
result(best_model_name, test_loss, test_acc, test_f1score, result_dict, best_time)

# Convert result_dict to a DataFrame
df = pd.DataFrame(result_dict)

# Convert any TensorFlow tensors in the F1Score column to numpy for better readability
df['Test F1Score'] = df['Test F1Score'].apply(lambda x: x.numpy() if hasattr(x, 'numpy') else x)

# Print the DataFrame as a table
print("Result Dictionary Table:")
print(df)

Results saved to results/result.csv
Result Dictionary Table:
                                          Model Name  Test Loss  \
0  EfficientnetV2B0 Without Augmentation Trainabl...   0.092373   
1  EfficientnetV2B3 Without Augmentation Trainabl...   0.053577   
2  EfficientnetV2M Without Augmentation Trainable...   0.122906   

   Test Accuracy  Test F1Score  Training Time  
0       0.985507      0.972536     284.886228  
1       0.992372      0.987055     818.727502  
2       0.982456      0.969305    4399.008303  


## Augmentation

In [19]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.25),
    tf.keras.layers.RandomTranslation(0.15, 0.15),
    # tf.keras.layers.RandomZoom(0.2, 0.2),
    # tf.keras.layers.RandomBrightness(0.2)
])

In [20]:
def augmentation_train(image, label):
    image = preprocess_input(image)
    image = data_augmentation(image)
    return image, label

def preprocess_val(image, label):
    image = preprocess_input(image)
    return image, label

In [ ]:
train_aug_300 = train_ds_300.map(augmentation_train, num_parallel_calls=tf.data.AUTOTUNE)
valid_aug_300 = valid_ds_300.map(preprocess_val, num_parallel_calls=tf.data.AUTOTUNE)

train_aug_300 = train_aug_300.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
valid_aug_300 = valid_aug_300.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
train_aug_300 = train_aug_300.concatenate(train_ds_300)

### EfficientNetV2B0

In [ ]:
model_name_1 = config_1.replace("Without", "with")
input_shape = (300,300,3)

aug_b0 = build_model_trainable_first25(EfficientNetV2B0, input_shape=input_shape, num_classes=4)
aug_b0, history_aug_b0, training_time_1 = train(aug_b0, model_name_1, train_aug_300, valid_aug_300, EPOCHS, SAVE_DIRECTORY)

In [ ]:
models = [
    (model_name_1, model_name_1, history_aug_b0, training_time_1)
]

best_model_name, test_loss, test_acc, test_f1score, best_time = best_variant(
    models=models,
    test_ds=test_ds_300,
    output_csv='EfficientNetV2B0 With Agumentation.csv',
    output_folder='results'
)

In [49]:
result(best_model_name, test_loss, test_acc, test_f1score, result_dict, best_time)

# Convert result_dict to a DataFrame
df = pd.DataFrame(result_dict)

# Convert any TensorFlow tensors in the F1Score column to numpy for better readability
df['Test F1Score'] = df['Test F1Score'].apply(lambda x: x.numpy() if hasattr(x, 'numpy') else x)

# Print the DataFrame as a table
print("Result Dictionary Table:")
print(df)


Results saved to results/result.csv
Result Dictionary Table:
                                          Model Name  Test Loss  \
0  EfficientnetV2B0 Without Augmentation Trainabl...   0.092373   
1  EfficientnetV2B3 Without Augmentation Trainabl...   0.053577   
2  EfficientnetV2M Without Augmentation Trainable...   0.122906   
3  EfficientnetV2B0 With Augmentation Trainable F...   0.133764   

   Test Accuracy  Test F1Score  Training Time  
0       0.985507      0.972536     284.886228  
1       0.992372      0.987055     818.727502  
2       0.982456      0.969305    4399.008303  
3       0.986270      0.974276     905.791257  


### EfficientNetV2B3

In [ ]:
model_name_2 = config_2.replace("Without", "With")
input_shape = (300,300,3)

aug_b3 = build_model_trainable_first25(EfficientNetV2B3, input_shape=input_shape, num_classes=4)
aug_b3, history_aug_b3, training_time_2 = train(aug_b3, model_name_2, train_aug_300, valid_aug_300, EPOCHS, SAVE_DIRECTORY)

In [ ]:
models = [
    (model_name_2, model_name_2, history_aug_b3, training_time_2)
]

best_model_name, test_loss, test_acc, test_f1score, best_time = best_variant(
    models=models,
    test_ds=test_ds_300,
    output_csv='EfficientNetV2B3 With Agumentation.csv',
    output_folder='results'
)

In [35]:
result(best_model_name, test_loss, test_acc, test_f1score, result_dict, best_time)

# Convert result_dict to a DataFrame
df = pd.DataFrame(result_dict)

# Convert any TensorFlow tensors in the F1Score column to numpy for better readability
df['Test F1Score'] = df['Test F1Score'].apply(lambda x: x.numpy() if hasattr(x, 'numpy') else x)

# Print the DataFrame as a table
print("Result Dictionary Table:")
print(df)

Results saved to results/result.csv
Result Dictionary Table:
                                           Model Name  Test Loss  \
0   EfficientnetV2B0 Without Augmentation Trainabl...   0.092373   
1   EfficientnetV2B0 Without Augmentation Trainabl...   0.092373   
2   EfficientnetV2B3 Without Augmentation Trainabl...   0.053577   
3   EfficientnetV2B0 Without Augmentation Trainabl...   0.092373   
4   EfficientnetV2B3 Without Augmentation Trainabl...   0.053577   
5   EfficientnetV2M Without Augmentation Trainable...   0.122906   
6   EfficientnetV2B0 Without Augmentation Trainabl...   0.092373   
7   EfficientnetV2B3 Without Augmentation Trainabl...   0.053577   
8   EfficientnetV2M Without Augmentation Trainable...   0.122906   
9   EfficientnetV2B0 With Augmentation Trainable F...   0.133764   
10  EfficientnetV2B3 With Augmentation Trainable F...   0.061761   

    Test Accuracy  Test F1Score  Training Time  
0        0.985507      0.972536     284.886228  
1        0.985507      0

### EfficientNetV2M

In [ ]:
model_name_3 = config_3.replace("Without", "With")
input_shape = (300,300,3)

aug_m = build_model_trainable_first25(EfficientNetV2B3, input_shape=input_shape, num_classes=4)
aug_m, history_aug_m, training_time_3 = train(aug_m, model_name_3, train_aug_300, valid_aug_300, EPOCHS, SAVE_DIRECTORY)

In [ ]:
models = [
    (model_name_3, model_name_3, history_aug_m, training_time_3)
]

best_model_name, test_loss, test_acc, test_f1score, best_time = best_variant(
    models=models,
    test_ds=test_ds_300,
    output_csv='EfficientNetV2M With Agumentation.csv',
    output_folder='results'
)

In [ ]:
result(best_model_name, test_loss, test_acc, test_f1score, result_dict, best_time)

# Convert result_dict to a DataFrame
df = pd.DataFrame(result_dict)

# Convert any TensorFlow tensors in the F1Score column to numpy for better readability
df['Test F1Score'] = df['Test F1Score'].apply(lambda x: x.numpy() if hasattr(x, 'numpy') else x)

# Print the DataFrame as a table
print("Result Dictionary Table:")
print(df)

### Combine All Results from Augmentation

In [ ]:
models = [
    (model_name_1, model_name_1, history_aug_b0, training_time_1),
    (model_name_2, model_name_2, history_aug_b3, training_time_2),
    (model_name_3, model_name_3, history_aug_m, training_time_3)
]

best_model_name, test_loss, test_acc, test_f1score, best_time = best_variant(
    models=models,
    test_ds=test_ds_300,
    output_csv='Agumentation.csv',
    output_folder='results'
)

# Result

In [ ]:
results = pd.DataFrame(result_dict)

print(results)

In [ ]:
results.to_csv("result.csv", index=False)

df = pd.read_csv("result.csv")

print(df)